---
### Util.

In [ ]:
import requests
from bs4 import BeautifulSoup
import pyspark as ps
import numpy as np
import time

response_cache = {}

def http_get(url):
    if response := response_cache.get(url):
        return response
    else:
        response = requests.request(url=url, method="GET")
        counties_list_html = response.content
        response_cache[url] = counties_list_html
        
        return response_cache[url]

spark = (ps.sql.SparkSession
         .builder
         .master('local[8]')
         .appName('lecture')
         .getOrCreate())

sc = spark.sparkContext

In [ ]:
def wikipedia_export_get(title):
    xml = http_get("https://en.wikipedia.org/wiki/Special:Export/" + title)
    
    if b'<redirect title' in xml:
        return wikipedia_export_get(title=BeautifulSoup(xml).select('redirect')[0]['title'])
    else:
        return xml

def wikipedia_standard_url(title):
    return "https://en.wikipedia.org/wiki/" + title


---

### Init!

In [ ]:
%%time

cities_all = (sc.textFile('communities_all.txt')
                .map(wikipedia_export_get)
                # .map(lambda export: BeautifulSoup(export).select('text')[0].getText())
                .cache())

cities_all.take(1)

In [ ]:
communities_all = (counties_export_text.map(wikipedia_communities_extract)
                                       .filter(lambda x: x != [])
                                       .map(lambda x: x[1]))

# out = [o for o in out if o[0] == tuple() and ',_Indiana' in o[1]]
# out = [o for o in out if o[0] != tuple() and ',_Puerto_Rico' not in o[1]]
# out = [o for o in out if o[0] != tuple() and ',_Indiana' in o[1]]
# out = [o for o in out if o[0] != tuple() and ',_Virginia' in o[1]]
# out = [o for o in out if o[0] != tuple() and ',_Alaska' in o[1]]

communities_all_1 = list(communities_all.map(set).reduce(set.union))

In [ ]:
from IPython.display import HTML
HTML('<br>'.join([f'<a href="{wikipedia_standard_url(title)}">{title}</a>' for title in np.random.choice(communities_all_1, 250)]))

---
#### Newtypes

In [ ]:
def test_1(x):
    with open('communities_all.txt') as f:
        return len(f.readlines()) + x

sc.parallelize(range(105)).map(test_1).collect()

---

#### Investigations

In [ ]:
print(BeautifulSoup(wikipedia_export_get(np.random.choice(communities_all_1))))